In [210]:
import pandas
from bs4 import BeautifulSoup
import regex
import numpy
import requests
import pprint

In [207]:
def strip_word(word: str) -> str:
    letters = []
    for i in word:
       if i.isalpha() or i.isspace():
            letters.append(i)
    word = ''.join(letters)
    return word.strip()


thi s


In [109]:
url = 'https://en.wikipedia.org/wiki/Major_League_Baseball_Most_Valuable_Player_Award'
html = requests.get(url)


In [134]:
soup = BeautifulSoup(html.content, 'html.parser')
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [214]:
tables = soup.select('.wikitable')[1:]
col_names = []
data = {}

for table in tables:
    headers = table.select('tr th')
    #Below logic gets header fields
    for header in headers:
        stripped_header = header.text.strip()
        if header.get('scope') == 'col' and stripped_header not in data.keys() and stripped_header !='Ref':
            data[stripped_header] = []
    #TODO:Add logic below on table variable to get data rows
    #Get all row values
    index = 0
    for row in table.select('tbody tr td, th'):
        if row.get('title') != '' and row.get('scope') != 'col':
            if index == 0:
                data['Year'].append(row.text.strip())
                index += 1
            elif index == 1:
                data['American League winner'].append(strip_word(row.text))
                index += 1
            elif index == 2:
                data['Team'].append(strip_word(row.text))
                index += 1
            elif index == 3:
                data['Position'].append(strip_word(row.text))
                index += 1
            elif index == 4:
                data['National League winner'].append(strip_word(row.text))
                index += 1
            elif index == 5:
                data['Team'].append(strip_word(row.text))
                index += 1
            elif index == 6:
                data['Position'].append((row.text))
                index += 1
            else:
                index = 0
        
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(data)

                
            
  


{ 'American League winner': [ 'Ty Cobb',
                              'Tris Speaker',
                              'Walter Johnson',
                              'Eddie Collins',
                              'George Sisler',
                              'Babe Ruth',
                              'Walter Johnson',
                              'Roger Peckinpaugh',
                              'George Burns',
                              'Lou Gehrig',
                              'Mickey Cochrane',
                              '',
                              'American League winner',
                              'Lefty Grove',
                              'Jimmie Foxx',
                              'Jimmie Foxx',
                              'Mickey Cochrane',
                              'Hank Greenberg',
                              'Lou Gehrig',
                              'Charlie Gehringer',
                              'Jimmie Foxx',
                            

In [203]:
print(data)

{'Year': ['1911\n', '1912\n', '1913\n', '1914\n'], 'American League winner': ['Ty Cobb\n', 'Tris Speaker\n', 'Walter Johnson\n', 'Eddie Collins\n'], 'Team': ['Detroit Tigers', 'Chicago Cubs', 'Boston Red Sox', 'New York Giants', 'Washington Senators', 'Brooklyn Superbas', 'Philadelphia Athletics', 'Boston Braves'], 'Position': ['OF\n', 'OF', 'OF\n', '2B', 'RHP\n', '1B', 'B\n', '2B'], 'National League winner': ['Frank Schulte\n', 'Larry Doyle\n', 'Jake Daubert\n', 'Johnny Evers\n']}


{'Year': [], 'American League winner': [], 'Team': [], 'Position': [], 'National League winner': []}
